# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. POOJA DILIP VERMA

2. MANDAR VILASRAO GITE

3. KAJARE BHUSHAN SHANTARAM SUVARNA

4. SINGH GANGASHANKAR RAJENDRAPRASAD INDRA


Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below`

In [ ]:
#Code Block : Set Initial State (Must handle dynamic inputs)

import math
import numpy as np

#State is structured as a dictionary where,
#First element - stock - is a 1D array describing the current food stock
#Second element - tents - is a 2D array describing the current outstanding food demand of each tent.
#Every row in 'tents' corresponds to one tent.

STOCK = 'stock'
TENTS = 'tents'

class State:
  def __init__(self, arr_stock, arr_tents, sid, cost):
    self.stock = arr_stock
    self.tents = arr_tents
    self.id = sid
    self.cost = cost

#Initializing data given in the problem statement
#All arrays specifying quantities of food items contain 5 elements, each denoting the quantity of a food item labelled as below:
#["Dhal", "Rice", "Flour", "Milk", "Bread"]

#Constants for food items
DAHL = 0
RICE = 1
FLOUR = 2
MILK = 3
BREAD = 4

MAX_QUANTITY = 15

# For Tent_1 to Tent _7 the requiremnt for a week in the tuple - (Dhal,Rice,Flour,Milk,Bread) i.e persons x per week requirement
tents_requirement = np.array([[3,9,9,6,2],
                              [12,36,36,0,0],
                              [2,6,6,9,3],
                              [4,12,12,0,0],
                              [3,9,9,6,2],
                              [2,6,6,3,1],
                              [2,6,6,9,3]]).astype(float)

In [ ]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import sys

# Action - One round trip of drone carrying specific no. of packets of one or more food items and
#       - Distributing the food packets for each food item in an equal ratio using the following rule such that,
#         In a trip, either tents with active demand get equal no. of packets, or min one packet if supply is less in any trip
# Path cost from state 1 to state 2 = One trip (1) + Undistributed food in state 2 post the trip

PACKET_SIZE = [2, 2.5, 3, 1, 1]
ID = 'id'
INITIAL_ID = 0
STATE = 'state'
COST = 'cost'

#Generate state-transition matrix for given initial state
#Transitions can be configured dynamically.
#By default, each state has 3 transitions based on the food items that can be carried in the next trip
# - MILK or BREAD or mix of DAHL, RICE, FLOUR
def generate_state_graph(initial):
  global initial_state
  initial_state = initial
  global id
  id = 0
  global state_graph
  state_graph = {}
  #Open list for unprocessed notes
  global open
  open = []

  print("id ", id)
  print("state_graph", state_graph)

  open.append(initial)

  while len(open) != 0:
    add_to_graph()


def add_to_graph():
  state = open.pop(0)

  if(is_goal_state(state)):
      return

  list_next_states = generate_next_states(state)
  state_graph[state.id] = list_next_states
  open.extend(list_next_states)

  print("For State ", state.id)
  print(state.stock)
  print(state.tents)

  for s in list_next_states:
    print(s.id, end=' ')
  print("Done")


def generate_next_states(state):
  if(state is None):
    return []

  possible_transitions = get_possible_transitions(state)
  list_new_states = []

  for transition in possible_transitions:
    new_state, leftover = get_new_state(state, transition)
    if(new_state is None):
      continue

    #print("For transition: ", transition)
    global id
    id += 1

    new_state.id = id
    print("New state id: ", new_state.id, new_state.cost)

    list_new_states.append(new_state)

  return list_new_states

import random
#Generates 1, 2 or all possible random transitions for experimental executon
def get_possible_transitions(state):
    global number_of_transitions

    pt = [[MILK], [BREAD], [DAHL, RICE, FLOUR]]

    #One random transition
    if number_of_transitions == 1:
      transition = None
      while(transition == None):
          i = random.randint(0,4)
          if(state.stock[i] != 0):
              transition = pt[i]

      print('transition: ', transition)
      return [transition]
    elif number_of_transitions == 2:
        transitions = None
        while(transitions == None):
            transitions = random.sample(range(0, 5), 2)
            print('transitions: ', transitions[0], transitions[1])
            return [[transitions[0]], [transitions[1]]]
    else:
      return pt

In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

#Get new state based on trip configuration
#param trip_config, 5 element array specifying the combination of food items to be carried in the transition trip
#param state, the state to transition from
#returns new state after transition
def get_new_state(state, trip_config):
  #print('Trip_config ', trip_config)
  can_create_state = False

  #Initialize new state
  new_state = get_deep_copy(state)
  stock = new_state.stock

  capacity = MAX_QUANTITY
  leftover = 0

  trip_items = np.zeros(5)
  for item in trip_config:
    if(capacity < PACKET_SIZE[item] or stock[item] == 0):
      continue

    can_create_state = True
    #1. Calculate the item quantity to be used in the trip
    q = min(capacity, stock[item])
    #No. of packets corresponding to q
    p = math.floor(q/PACKET_SIZE[item])
    q = p*PACKET_SIZE[item]

    trip_items[item] = q
    capacity = capacity - q

    #2. Update new state with demands met from packet distribution using the defined distribution rule
    item_leftover = update_demand(item, p, new_state.tents)
    leftover = leftover + item_leftover

    #Update stock. Limit to 0 for depletion
    stock[item] = max(0, stock[item] - q + item_leftover)
    new_state.cost = calc_path_cost(trip_items, leftover)

  if(can_create_state == True):
    return new_state, leftover
  else:
    return None, 0

def get_deep_copy(state):
  arr_stock = np.copy(state.stock)
  arr_tents = np.copy(state.tents)

  return State(arr_stock, arr_tents, -1, 0)

def update_demand(item, p, tents):
  #Calculate the no. of packets to be distributed, per tent with non-zero demand, for this food item
  ppt = get_packets_per_tent(item, p, tents)

  leftover = p
  for tent in tents:
    #Distribute until cargo is not over
    if(leftover == 0):
      break

    demand = tent[item]
    if(demand == 0):
      continue

    #Handle packet per tent comes to 0 due to floor calculation
    if(ppt == 0 and demand != 0):
      to_give = 1
    else:
      demand_in_packets = math.ceil(demand/PACKET_SIZE[item])
      to_give = min(ppt, demand_in_packets)

    #Update pending demand for the tent
    demand = max(0, demand - (to_give * PACKET_SIZE[item]))

    tent[item] = demand
    leftover = leftover - to_give

  return (leftover * PACKET_SIZE[item])


def get_packets_per_tent(item, packets, tent_state):
  #Get tents with non-zero demand
  n = sum(tent_state[:, item] > 0)

  if(n == 0):
    return 1

  ppt = math.floor(packets/n)
  return ppt

TRIP_COST = 1
#Calculate path cost for trip carrying given items and leftover penalty
def calc_path_cost(trip_items, leftover):
  cost = (TRIP_COST * np.sum(trip_items)) + leftover
  return cost


In [ ]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

#Goal state is achieved if entire stock is over
def is_goal_state(state):
  return not np.any(state.stock)

### 2.	Definition of Algorithm 1 (A* Search algorithm)

In [ ]:
#Code Block : Function for A* Search implementation

def a_star(initial_state):
    iid = initial_state.id

    open_set = set()
    open_set.add(initial_state)
    #Placeholder to track size for space complexity observation
    open_set_size = len(open_set)

    closed_set = set()
    g = {}               #Store distance from starting node
    parents = {}         #Parents contains an adjacency map of all nodes
    #Distance of starting node from itself is zero
    g[iid] = 0
    #Start_node is root node i.e it has no parent nodes, so start_node is set to its own parent node
    parents[iid] = iid

    while len(open_set) > 0:
        current = None
        #Node with lowest f() is found
        for state in open_set:
            if current == None or g[state.id] + heuristic(state) < g[current.id] + heuristic(current):
                current = state
                print("Pop from Open: ", current.id, current.stock, current.tents, current.cost, g[current.id], heuristic(current))

        if is_goal_state(current) or state_graph[current.id] == None:
            pass
        else:
            for neighbor in get_neighbors(current.id):
                #Nodes not in first and last set are added to first
                if neighbor not in open_set and neighbor.id not in closed_set:
                  open_set.add(neighbor)
                  open_set_size += 1
                  parents[neighbor.id] = current.id
                  g[neighbor.id] = g[current.id] + neighbor.cost
                  print("Adding to Open set: ", neighbor.id, neighbor.cost, g[neighbor.id], heuristic(neighbor))
                else:
                  #For each neighbor, compare its distance from start i.e g(m) to the #from start through n node
                  if g[neighbor] > g[current.id] + neighbor.cost:
                      #update g(m)
                      g[neighbor] = g[current.id] + neighbor.cost
                      #change parent of m to n
                      parents[neighbor] = current.id
                      #if m in closed set,remove and add to open
                      if neighbor.id in closed_set:
                          closed_set.remove(neighbor.id)
                          open_set.add(neighbor)
                          open_set_size += 1

        if current == None:
            print('Path does not exist!')
            return None

        # if the current node is the stop_node
        # then we begin reconstructin the path from it to the start_node
        if is_goal_state(current):
            path = []
            cid = current.id
            while parents[cid] != cid:
                path.append(cid)
                cid = parents[cid]
            path.append(iid)
            path.reverse()
            print('Path found: {}'.format(path))
            print('Path cost: ', g[current.id])
            print('Size of open set:', open_set_size)
            print('Size of closed set:', len(closed_set))
            print('No. of trips required: ', len(path)-1)
            return path
        # remove n from the open_list, and add it to closed_list
        # because all of his neighbors were inspected
        open_set.remove(current)
        closed_set.add(current.id)
    print('Path does not exist!')
    return None

def get_neighbors(sid):
    if sid in state_graph:
        return state_graph[sid]
    else:
        return None

#Heauristic of a state is the amount of stock left to be distributed
def heuristic(state):
    return np.sum(state.stock)

### 3.	Definition of Algorithm 2 (Genetic Algorithm)

In [ ]:
!pip install bitarray
import numpy as np
import random
import itertools
from bitarray import bitarray

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


MUTATE_FLAG = False
# MAX_ITER = 10
MAX_ITER = 1000
# MAX_ITER = 100
# MAX_ITER = 10000


INITIAL_FITNESS = 120

NUM_PLANS =100

NUM_TRIPS = 50
w_1 =1.0
w_2 =1.0
w_3 =0.0

# Initialise arbitrary high values for the paramters of fitnes function to be minimized
max_trips,inital_variance, initial_undistributed = 140,1000,154


# Items in supply
items = np.array([(1,15,2,30),
				 (2,20,2.5,50),
				 (3,12,3,36),
				 (4,28,3,28),
				 (5,10,1,10)])

items = items.astype(float)
godown = items[:,1].astype(float)



# persons and their weekly requirement
persons = np.array([('Adult', 1,3,3,0,0),
				   ('Children',0,0,0,3,1)
				   ])

# no of Adults and Children in Tent1 to Tent7
tents_people =np.array([(3,2),
						(12,0),
						(2,3),
						(4,0),
						(3,2),
						(3,2),
						(2,3)
						])
# For Tent_1 to Tent _7 the requiremnt for a week in the tuple - (Dhal,Rice,Flour,Milk,Bread) i.e persons x per week requirement
tents_requirement = np.array([(3,9,9,6,2),
							  (12,36,36,0,0),
							  (2,6,6,9,3),
							  (4,12,12,0,0),
							  (3,9,9,6,2),
							  (3,9,9,6,2),
							  (2,6,6,9,3),])



demand_matrix = [
		[1.0, 2.0, 1.0, 6.0, 2.0],
		[7.0, 9.0, 4.0, 0.0, 0.0],
		[1.0, 1.0, 1.0, 7.0, 3.0],
		[3.0, 3.0, 2.0, 0.0, 0.0],
		[1.0, 3.0, 2.0, 5.0, 2.0],
		[1.0, 1.0, 1.0, 4.0, 1.0],
		[1.0, 1.0, 1.0, 6.0, 2.0]
	]


weekly_requirement = np.array([[1,3,3,0,0],
				   [0,0,0,3,1]])

# Calculate Tent_Requirement_kg
Tent_Requirement_kg = np.matmul(tents_people, weekly_requirement)



# Create the drone_parcel_limit Min and Max no of Parcels
drone_parcel_limit = np.array([[0, 0, 0, 0, 0], [7, 6, 5, 15, 10]])

drone_capacity_limit = 15



# Calculate the Kgs in a parcel
kg_in_parcel = items[:, 2].astype(float)

mutation_rate =0.05


# define a function to convet parcel to kg
def convert_parcel_to_kg(num_parcels, items=items):

	# kg_in_parcel = items[:, 2].astype(float)
	return num_parcels*kg_in_parcel

# define a function to calcualte the per person distributed items
def calculate_perperson(tents_supplied, tents_people):

	#Calculate the average number of items supplied per adult in each tent
	adult_supplied = tents_supplied[:, :3]
	adult_people = tents_people[:, 0][:, np.newaxis]
	adult_average = np.where(adult_people != 0, adult_supplied / adult_people, 0)

	# Calculate the average number of items supplied per child in each tent
	child_supplied = tents_supplied[:, 3:]
	child_people = tents_people[:, 1][:, np.newaxis]
	child_average = np.where(child_people != 0, child_supplied / child_people, 0)

	# Concatenate the adult_average and child_average arrays horizontally
	tents_supplied_average = np.hstack((adult_average, child_average))

	return tents_supplied_average


# define a function to calculate column wise ( items wise ) variance of supplied item distribution profile in kg
def compute_variance(array):
	# Calculate the variance along axis 0 (columns)
	column_variances = np.var(array, axis=0)
	return column_variances.sum()

def compute_trips(solution):


	num_trips_taken = np.sum(np.any(solution != 0, axis=1))

	return num_trips_taken

def compute_godown(array):
	sliced_array = array[:, :5]
	summed_columns = np.sum(sliced_array, axis=0)
	result = summed_columns - godown
	return result
def tents_stock(solution):
	"""
	Calculate the commodities stock in each tent and the total commodities stock across all tents.

	Args:
		solution (SortiePlan): An instance of the SortiePlan class containing population data.

	Returns:
		tuple: A tuple containing two values:
			- total_stock (np.ndarray): A 1D numpy array representing the summed up commodities stock
									   across all tents for each commodity type.
			- stocks_per_tent (list of np.ndarray): A list where each element is a 1D numpy array
													 representing the commodities stock for a specific tent.
	Raises:
		ValueError: If the provided solution is not an instance of SortiePlan.
	"""
	# if not isinstance(solution, SortiePlan):
	# 	raise ValueError("tents_stock takes an instance of SortiePlan as an argument")

	stocks_per_tent = []
	total_stock = np.zeros(5)

	for i in range(7):
		stock_in_tent = np.sum(solution[:, i * 5+5 : (i + 2) * 5], axis=0)
		stocks_per_tent.append(stock_in_tent)
		total_stock += stock_in_tent

	stocks_per_tent_array = np.array(stocks_per_tent) # Convert to NumPy array

	return total_stock, stocks_per_tent_array




# define a function to compute the cargo parcels left undistributed and report if its zero less than zero or more than zero
def compute_undistributed(solution,items=items):

	"""'''
	Input:
	solution: a numpy array of the solution ( yellow box)
	items: a numpy array of the items ( godown of stock at the start state )

	Output:
	undistributed_items: a float of the number of undistributed items ( Total no os undistributed items)
	'''"""

	# pick up the first five columns of solution
	parcels= solution[:, :5]

	parcels_sum = np.sum(parcels,axis=0)

	undistributed_items = items[:, 1].astype(float) - parcels_sum

	return np.sum(undistributed_items)

def compute_fitness(solution,w_1=w_1,w_2=w_2,w_3=w_3):
	# Summarize the distribution post the population
	tent_distributed_parcels = np.array([np.sum(solution[:, i:i + 5], axis=0) for i in range(5, 40, 5)])

	# Convert the distribution in terms of kg's
	tent_distributed_kg = convert_parcel_to_kg(tent_distributed_parcels)

	# Compute the variance for this distribution
	variance = compute_variance(tent_distributed_kg)

	# Compute the number of rows with non-zero entries
	non_zero_count = np.sum(np.any(solution != 0, axis=1))

	# compute the undistributed parcels for a population
	undistributed = compute_undistributed(solution)

	# Calculate the fitness value
	fitness = w_1*non_zero_count + w_2*variance + w_3*undistributed

	return fitness






def update_godown(drone_load: int, godown: np.ndarray=np.array(items[:,1])) -> np.ndarray:
	'''Utility function to update godown stock after a trip ( row of the yellow box) is completed.

	param : drone_load (int) - number of units delivered by the drone
	param : godown (np.ndarray) - current godown stock
	return : updated_godown (np.ndarray) - updated godown stock

	'''

	# if not isinstance(drone_load, np.int32):
	#     raise ValueError("drone_load should be an instance of numpy.int32")
	# print("The passed godwon has type ",type(godown))
	if godown.shape != (5,):
		raise ValueError("godown should be a numpy array of shape (5,)")

	updated_godown = godown.copy()
	# print(f" Godown in the {type(updated_godown)} and drone_laod has type{type(drone_load)}")

	updated_godown -= drone_load

	# Ensure all quantities are non-negative
	updated_godown[updated_godown < 0] = 0

	return updated_godown


def satisfies_constraints(drone_load, items=items, drone_parcel_limit=drone_parcel_limit[1],kg_in_parcel=kg_in_parcel):


	# If Bread is selected , then it be less than parcel limit of 10 and  other items are zero
	if drone_load[4] > 0 and (drone_load[0] > 0 or drone_load[1] > 0 or drone_load[2] > 0 or drone_load[3] > 0):
		return False

	#  if Milk is selected i.e  drone_load[3] > 0, then all other should be zero then parcel limit is 15
	if drone_load[3] > 0 and (drone_load[0] > 0 or drone_load[1] > 0 or drone_load[2] > 0):
		return False

	# if Dhal, Rice and Flour is selected then there combined weight should be less than fifteen  and Bread or Milk should be zero
	kg_in_parcel = items[:, 2].astype(float)
	if any(drone_load[0:2])>0 and (drone_load[0]*kg_in_parcel[0] + drone_load[1]*kg_in_parcel[1] + drone_load[2]*kg_in_parcel[2]) > 15:
		return False

	# # Check if each column in 'drone_load' is equal to the corresponding value in the last column of 'items'
	if not np.all(drone_load[:5] <= drone_parcel_limit[1]):
		return False


	# if np.sum(drone_load) > items[1]:
		# return False

	# If all constraints are satisfied
	return True


def plan_trips(num_trips, items = items):

	""" Use this function to generate a random set of trips.
		Each trip is a row in the array with  5-element array with the following format:
		[item1, item2, item3, item4, item5]
		where each element is the number of units to be delivered across 7 tents

		param : num of trips in the plan (int)
		return : a numpy array of shape (num_trips, 5)
	"""

	# Initialize an array with the number of trips as rows and DRFMB as columns
	planned_trips = np.zeros((num_trips, 5), dtype=int)

	# Set max limit for each column
	updated_godown = np.array(items[:,1])

	for i in range(num_trips):

		max_limits = np.minimum(updated_godown, drone_parcel_limit[1])

		# max_limits= updated_godown
		# print(max_limits)
		trip_generated= False
		while trip_generated == False:
			for j in range(5):

				# Check if max_limit for the current item is greater than zero

				if max_limits[j] > 0:

					# Populate the current cel  l with random integer up to max_limit
						planned_trips[i,j] = np.random.randint(0, max_limits[j] + 1)

				else:

					# If max_limit for the current item is not greater than zero, assign zero to the current cell
					planned_trips[i,j] = 0


			variable= satisfies_constraints(planned_trips[i,:5])


			if variable:
				trip_generated = True


		updated_godown = update_godown(planned_trips[i,:5],updated_godown)


		if np.all(updated_godown == 0):
			break



	return planned_trips,updated_godown


class SortiePlan:

	def __init__(self):
		self.population = None
		self.fitness = None
		self.survival = None
		self.leftover = None
		self.max_trips = None
		self.trips_required = None
		self.stock_godown = None
		self.tents_stock = None
		self.variance = None

	def update_values(self, population, fitness, leftover, max_trips, trips_required, tents_stock, variance,godown):
		self.population = population
		self.fitness = fitness
		self.leftover = leftover
		self.max_trips = max_trips
		self.trips_required = trips_required
		self.tents_stock = tents_stock
		self.variance = variance
		self.stock_godown = godown


	def __repr__(self):
		return (
			f"Population: {self.population}, "
			f"Fitness: {self.fitness}, "
			f"Survival: {self.survival}, "
			f"Leftover: {self.leftover}, "
			f"Max Trips: {self.max_trips}, "
			f"Trips Required: {self.trips_required}, "
			f"Stock Godown: {self.stock_godown}, "
			f"Tents Stock: {self.tents_stock}, "
			f"Variance: {self.variance}"
		)

	def __str__(self):
		return (
			f"Population: {self.population}, "
			f"Fitness: {self.fitness}, "
			f"Survival: {self.survival}, "
			f"Leftover: {self.leftover}, "
			f"Max Trips: {self.max_trips}, "
			f"Trips Required: {self.trips_required}, "
			f"Stock Godown: {self.stock_godown}, "
			f"Tents Stock: {self.tents_stock}, "
			f"Variance: {self.variance}"
		)

	def __getattr__(self):
			raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")


def design_solution(drone_load):
	"""
	Generate a solution for distributing commodities among tents based on the provided drone load. ( Right side of yellow box  )

	Args:
		drone_load (np.ndarray): A 2D numpy array representing the commodities load for each drone, where each row
								corresponds to a drone's load in terms of different commodities.

	Returns:
		np.ndarray: A 2D numpy array representing the designed solution, where commodities are distributed among tents.
					The first 5 columns represent the drone load, and the rest represent the tent distribution.
	"""
	# Instantiating the arrays
	solution_derived = np.zeros((drone_load.shape[0], 40))
	tents = [np.zeros((drone_load.shape[0], 5)) for _ in range(7)]

	demand_matrix = [
		[1.0, 2.0, 1.0, 6.0, 2.0],
		[7.0, 9.0, 4.0, 0.0, 0.0],
		[1.0, 1.0, 1.0, 7.0, 3.0],
		[3.0, 3.0, 2.0, 0.0, 0.0],
		[1.0, 3.0, 2.0, 5.0, 2.0],
		[1.0, 1.0, 1.0, 4.0, 1.0],
		[1.0, 1.0, 1.0, 6.0, 2.0]
	]


	# The sum of columns constraints
	godown = np.array([15.0, 20.0, 12.0, 28.0, 10.0])

	#create copy of demand matrix
	dm = np.copy(demand_matrix)

	for row in range(drone_load.shape[0]):
		current_row = drone_load[row]

		is_balanced =False
		while is_balanced ==False:
			current_copy = np.copy(current_row)

			for tent in tents:
				tent[row,:]=0

			for col in range(5):
				if current_copy[col] ==0:
					continue

				for tent_num, tent in enumerate(tents):


					if current_copy[col]==0:
						break

					max_limit = min(demand_matrix[tent_num][col], current_copy[col])

					if max_limit <=0 :
						continue


					random_num = np.random.randint(1, max_limit + 1)

					tent[row, col] = random_num

					current_copy[col] -= tent[row, col]


				# This is for extra cargo ( more than total demand)
				if current_copy[0]!=0:
					tents[1][row,0]=current_copy[0]
					current_copy[0] -= tents[1][row,0]


			sum_tents= np.zeros((drone_load.shape[0], 5))
			i=0
			for tent in tents:
				sum_tents+=tent
				i+=1
#

			temp = sum_tents[row,:]
			check = False
			counter = 0
			for temp_num, temp in enumerate(temp):
				if drone_load[row,temp_num] == temp:
					counter+=1
					continue
				else:
					break
			if counter==5:
				check = True
				is_balanced = True
#



		for tent_num, tent in enumerate(tents):
			demand_matrix[tent_num]-= tent[row]

		printing_demand_matrix = np.array(demand_matrix)


	# Transposing the tents, stacking horizontally, and flattening
	transposed_tents = [tent.T for tent in tents]
	stacked_tents = np.hstack(tents)

	flattened_tents = stacked_tents.flatten()

	# Adding to solution_derived
	solution_derived[:, 5:40] = flattened_tents.reshape(drone_load.shape[0], 35)

	# Adding the drone_load to the first 5 columns of solution_derived
	solution_derived[:, :5] = drone_load

	solution = SortiePlan()
	solution.population = solution_derived




	return solution_derived #solution

def generate_plans(plan_size : int =10,):
	""" function to generate a set of solutions ( i.e yello boxes ) consisting of  = plan_size
		Input:
			no of solutions to be generated ( no of yellow boxes)
			no of trips in each solution  (no of rows in yellow boxes)
		Output:
			a list of SortiePlan objects


	"""
	global NUM_TRIPS

	plans = [SortiePlan() for i in range(plan_size)]


	for plan in plans:

		set_of_trips = plan_trips(NUM_TRIPS)
		population = design_solution(set_of_trips[0])
		fitness = compute_fitness(population)
		leftover = compute_undistributed(population)
		max_trips = population.shape[0]
		trips_required = compute_trips(population)
		tents_stock_values = tents_stock(population)[1]
		variance = compute_variance(tents_stock_values)
		godown = compute_godown(population)

		plan.update_values(population, fitness, leftover, max_trips, trips_required, tents_stock_values, variance,godown)




	summation_fitness =0

	for plan_num, plan in enumerate(plans):
		# print(f"checking {plan_num}")
		# print(plan.fitness)
		summation_fitness = summation_fitness + plan.fitness

	for plan_num, plan in enumerate(plans):
		plan.survival = plan.fitness / (summation_fitness * 1.0)

		summation_fitness = summation_fitness + plan.fitness




	return plans



def get_parent():
	''' function to select parents from the list of solutions (SortiePlan) based on survival value
		Input: none
		Output: Two instances of class SortiePlans
	'''

	parent1, parent2 = None, None

	while True:
		parent1_random = np.random.rand()
		parent1_rn = [x for x in ini_pop if x.survival < parent1_random]
		try:
			parent1 = parent1_rn[0]
			break
		except:
			pass

	while True:
		parent2_random = np.random.rand()
		parent2_rn = [x for x in ini_pop if x.survival <= parent2_random]
		try:
			t = np.random.randint(len(parent2_rn))
			parent2 = parent2_rn[t]
			if parent2 != parent1:
				break
			else:
				continue
		except:
			continue

	if parent1 is not None and parent2 is not None:
		return parent1, parent2
	else:
		sys.exit(-1)



def reproduce_crossover(parent1, parent2):
	''' function receives two solutions ( two yellow boxes)  as two parents  and returns two children
		Input: 2 solutions ( Isntances of SortiePlan class)
		output: 2 solutions (Instances of SortiePlan class)

	'''
	# Define Grains and Dairy arrays
	grains = np.array([0, 1, 2])
	dairy = np.array([3, 4])

	pop1 = parent1.population[:,:5]
	pop2 = parent2.population[:,:5]


	# Select rows from parent1 where the specified columns are not zero
	rows_parent1_grains = pop1[np.any(pop1[:, grains] != 0, axis=1)]
	rows_parent1_dairy = pop1[np.any(pop1[:, dairy] != 0, axis=1)]

	# print(rows_parent1_grains.shape)
	# print(rows_parent1_dairy.shape)


	# Select rows from parent2 where the specified columns are not zero
	rows_parent2_grains = pop2[np.any(pop2[:, grains] != 0, axis=1)]
	rows_parent2_dairy = pop2[np.any(pop2[:, dairy] != 0, axis=1)]

	# print(rows_parent2_grains.shape)
	# print(rows_parent2_dairy.shape)


	# Concatenate the selected rows from parent1 and parent2
	selected_rows_1 = np.vstack((rows_parent1_grains, rows_parent2_dairy))
	selected_rows_2 = np.vstack((rows_parent2_grains,rows_parent1_dairy))

	# Initialize the child  array
	child1 = np.zeros((100, 5))

	child1[:selected_rows_1.shape[0],:selected_rows_1.shape[1]]= selected_rows_1

	#put in all rows zero  in child 1

	# Initialize the child  array
	child2 = np.zeros((100, 5))

	child2[:selected_rows_2.shape[0],:selected_rows_2.shape[1]]= selected_rows_2


	# #put in all rows zero  in child 2


	# print(child1.shape)
	# print(child2.shape)



	return child1, child2
def binary_encoding(child):
	max_value = int(np.max(child))
	num_bits_per_element = int(np.ceil(np.log2(max_value + 1)))

	# Initialize an empty binary matrix
	binary_matrix = np.zeros((child.shape[0], child.shape[1] * num_bits_per_element), dtype=int)

	# Convert each element in the array to binary and populate the binary matrix
	for row in range(child.shape[0]):
		for col in range(child.shape[1]):
			binary_value = format(int(child[row, col]), f'0{num_bits_per_element}b')
			binary_matrix[row, col * num_bits_per_element: (col + 1) * num_bits_per_element] = [int(bit) for bit in binary_value]

	return binary_matrix


# convert binary into integer
def binary_decoding(binary_population):
	n_rows, n_cols = binary_population.shape
	num_bits = n_cols // 4

	decoded_population = np.zeros((n_rows, num_bits), dtype=int)

	for i in range(n_rows):
		for j in range(num_bits):
			# Convert 4-bit binary representation to integer value
			binary_array = bitarray(binary_population[i, j*4:(j+1)*4].tolist())
			decoded_population[i, j] = int(binary_array.to01(), 2)

	return decoded_population


def mutate(child, mutation_rate=0.05):
	 # Encode child.population in binary
	child_binary = binary_encoding(child)

	# Calculate the number of bits to mutate based on mutation_rate
	num_bits_to_mutate = int(mutation_rate * child_binary.size)

	# Generate random positions to mutate (random integers between 0 and the total number of bits for each element)
	num_elements, num_bits_per_element = child_binary.shape
	mutation_positions = np.random.randint(0, num_bits_per_element, num_bits_to_mutate)

	# Iterate over the mutation positions and flip the bits
	for row in range(num_elements):
		for pos in mutation_positions:
			# Generate a random number between 1 and 100
			random_number = np.random.randint(1, 101)

			# Mutate the cell with a probability proportional to the random_number
			if random_number <= 20:
				# Flip the bit at the mutation position
				child_binary[row, pos] = not child_binary[row, pos]

	# Decode the mutated binary back to integer
	child = binary_decoding(child_binary)

	return child


def GA(iteration):



	if iteration%50==0:
		print("########## Executing Genetic Generation:", iteration, "##########")


#    print(f"Trying to prodcuce cross over :{i}")
	new = []
	new_pop = []

	# print(f"length of new pop  at the start of loop is {len(new)}")


	for i in range(len(ini_pop)):


		parent1, parent2 = get_parent()



		child1, child2 = reproduce_crossover(parent1, parent2)

		new.append(child1)
		new.append(child2)


		if (MUTATE_FLAG):
			print("Trying to Mutate")
			for each in new:
				presentVal = each


				nothing_left = False
				while nothing_left==False:
					mightBeChangedVal = mutate(each)

					if compute_undistributed(mightBeChangedVal)== 0:
						print(f"mutation happened")
						nothing_left = True

					else:
						print(f"mutation did not happen")


				if not np.array_equal(presentVal, mightBeChangedVal):
					each = mightBeChangedVal





	new_pop=[]
	for each in new:
		# each.survival = each.fitness / (summation_fitness * 1.0)

		set_of_trips = each[:,:5]

		population = design_solution(set_of_trips)
		fitness = compute_fitness(population)
		leftover = compute_undistributed(population)
		max_trips = population.shape[0]
		trips_required = compute_trips(population)
		tents_stock_values = tents_stock(population)[1]
		variance = compute_variance(tents_stock_values)
		godown = compute_godown(population)

		each= SortiePlan()
		each.update_values(population, fitness, leftover, max_trips, trips_required, tents_stock_values, variance,godown)
		new_pop.append(each)

	summation_fitness = np.sum([x.fitness for x in new_pop])

	for each in new_pop:

		each.survival = each.fitness / (summation_fitness * 1.0)



	return new_pop



def print_output(final_pop):

	print("No of Solutions obtained:",len(final_pop))
	print("------------------------")
	result=[]

	subopt_results =[]

	non_feasible_results = []

	for i in range(0, len(final_pop)):

		final_pop[i].leftover = compute_undistributed(final_pop[i].population)
		# print(f"Left over for {i}th solution :{test[i].leftover}")
		if final_pop[i].leftover==0:
			result.append(final_pop[i])

		if final_pop[i].leftover> 0:
			subopt_results.append(final_pop[i])

		if final_pop[i].leftover < 0 :
			non_feasible_results.append(final_pop[i])



	sorted_subopt = sorted(subopt_results, key=lambda x: x.leftover)


	# Ensure there are at least three solutions, else print as many as available
	num_to_print = min(4, len(sorted_subopt))


	print("No of Non Feasible Solutions :",len(non_feasible_results))
	print("---------------------------------------")
	print("No of Feasible Solutions :",len(subopt_results)+len(result))
	print("---------------------------------------")

	print(f"Solution No 1:")

	print("   Undistributed Parcels:", 0)

	#find the lowest fitness score in result
	min_element = min(result, key=lambda x: x.fitness)

	print_solution_info(min_element)

	return min_element


def print_solution_info(solution):


	population = solution.population
	survival = solution.survival
	fitness = solution.fitness
	leftover = solution.leftover

   	# print("Stock in Godown")

	# quantities = items[:,1:2]

	# print(quantities.T)

	print("----------------")

	print("Stock Carried by Drone")

	print(np.sum(solution.population[:,:5], axis=0).astype(float))

	print("----------------")
	print("Stock Recieved in Tents")

	print(solution.tents_stock)



	print("----------------")
	print("Tent Demand Matrix")
	print(np.array(demand_matrix))



	print("-----------------------------")

	print("No of Trips  : " , solution.trips_required)
	print("Variance:", solution.variance)
	print("Fitness : ",fitness)



def tabulate(plans):

	# Create DataFrame from the list of dictionaries
	plans_data = [{
		'Fitness': plan.fitness,
		'Leftover': plan.leftover,
		'Max Trips': plan.max_trips,
		'Trips Required': plan.trips_required,
		'Stock Godown': plan.stock_godown,
		'Tents Stock': plan.tents_stock,
		'Variance': plan.variance
	} for plan in plans]

	df = pd.DataFrame(plans_data)

	df = df[['Fitness', 'Trips Required', 'Variance']]

	sns.lineplot(x='Trips Required', y='Variance', data=df)

	plt.show()
	return df



def stop(iteration):
	# Access the global variable for initial_fitness and MAX_ITER
	global INITIAL_FITNESS, MAX_ITER

	initial_fitness = INITIAL_FITNESS

	# Check if any plan's fitness is less than the initial_fitness
	fitness_values = [plan.fitness for plan in ini_pop]

	if any(fitness < INITIAL_FITNESS for fitness in fitness_values):
		initial_value = min(fitness_values)

	# Check if the maximum number of iterations is reached
	if MAX_ITER == iteration:
		return True


	# If none of the stopping conditions are met, return False to continue the loop
	return False


# Run the GA process until the stop condition is met
def run_GA():

	test = generate_plans(NUM_PLANS)

	iteration = 0
	while not stop(iteration):
	# new_pop=[]
		test = GA(iteration)
		iteration += 1

	print_output(test)

	return



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 5.0 MB/s eta 0:00:00


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)
# For A* Alogorithm
#initial = State(supply, tents_requirement, 0, 0)

#End state is checked by calling the function is_goal_state(state)


# For Genetic Algorithm the end state when the  when the tent recieved matrix is same as demand matrix.

# Demand Matrix in the present code is designed with minimum variance as the goal state. i.e Minimim No of Trips to ensure equitable distribution to extent possible



### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
import timeit

def invoke_a_star(root):
    #Start the timer
    startTime = timeit.default_timer()

    a_star(root)

    endTime = timeit.default_timer()
    timetaken = endTime - startTime

    #print("Time taken: ", timetaken)
    return timetaken

# Invoking A* for the Problem statement
supply = [30, 50, 36, 28, 10]
tents_requirement = np.array([[3,9,9,6,2],
                              [12,36,36,0,0],
                              [2,6,6,9,3],
                              [4,12,12,0,0],
                              [3,9,9,6,2],
                              [2,6,6,3,1],
                              [2,6,6,9,3]]).astype(float)

number_of_transitions = 3
initial = State(supply, tents_requirement, 0, 0)
#Step 1: Generate State graph
generate_state_graph(initial)

Streaming output truncated to the last 5000 lines.
 [ 0.   0.   0.   2.   0. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   0. ]
 [ 0.   0.   6.   0.   0. ]
 [ 0.   6.   6.   3.   1. ]]
75620 Done
New state id:  75621 1.0
For State  55601
[0 0 0 1 0]
[[ 0.   0.   0.   0.   0. ]
 [ 0.  26.  27.   0.   0. ]
 [ 0.   0.   0.   3.   0. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   0. ]
 [ 0.   0.   6.   0.   0. ]
 [ 0.   6.   6.   3.   1. ]]
75621 Done
New state id:  75622 1.0
For State  55602
[0 0 0 1 0]
[[ 0.   0.   0.   0.   0. ]
 [ 0.  26.  27.   0.   0. ]
 [ 0.   0.   0.   3.   0. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   0. ]
 [ 0.   0.   6.   0.   0. ]
 [ 0.   6.   6.   3.   1. ]]
75622 Done
New state id:  75623 6.0
For State  55603
[0 0 6 0 0]
[[ 0.   0.   3.   0.   0. ]
 [ 0.  26.  30.   0.   0. ]
 [ 0.   0.   0.   2.   0. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   0. ]
 [ 0.   0.   6.   0.   0. ]
 [ 0.   6.   6.   3.   1. ]]
75623 Done
New state i

In [ ]:
#Step 2: Execute the algorithm
invoke_a_star(initial)

Streaming output truncated to the last 5000 lines.
 [ 0.   0.   0.   3.   3. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   2. ]
 [ 0.   0.   6.   0.   1. ]
 [ 0.   6.   6.   3.   3. ]] 6.0 157.0 11
Adding to Open set:  30841 1.0 158.0 10
Adding to Open set:  30842 11.0 168.0 2
Pop from Open:  50874 [0 0 0 1 0] [[ 0.   0.   0.   0.   0. ]
 [ 0.  26.  27.   0.   0. ]
 [ 0.   0.   0.   3.   0. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   0. ]
 [ 0.   0.   6.   0.   0. ]
 [ 0.   6.   6.   3.   1. ]] 14.0 169.0 1
Pop from Open:  18549 [ 0  0  6  0 10] [[ 0.   0.   3.   0.   2. ]
 [ 0.  26.  30.   0.   0. ]
 [ 0.   0.   0.   2.   3. ]
 [ 0.   4.5  6.   0.   0. ]
 [ 0.   1.5  3.   0.   2. ]
 [ 0.   0.   6.   0.   1. ]
 [ 0.   6.   6.   3.   3. ]] 15.0 152.0 16
Adding to Open set:  30827 11.0 163.0 7
Adding to Open set:  30828 6.0 158.0 10
Pop from Open:  50874 [0 0 0 1 0] [[ 0.   0.   0.   0.   0. ]
 [ 0.  26.  27.   0.   0. ]
 [ 0.   0.   0.   3.   0. ]
 [ 0.   4.5  6.   0.   

150.3751909980001

In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

ini_pop = generate_plans(NUM_PLANS)
print("-----------")
print(f"Initial Population Generated of size : {len(ini_pop)}")
test = run_GA()

print("Thank You!")

-----------
Initial Population Generated of size : 100
########## Executing Genetic Generation: 0 ##########
########## Executing Genetic Generation: 50 ##########
########## Executing Genetic Generation: 100 ##########
########## Executing Genetic Generation: 150 ##########
########## Executing Genetic Generation: 200 ##########
########## Executing Genetic Generation: 250 ##########
########## Executing Genetic Generation: 300 ##########
########## Executing Genetic Generation: 350 ##########
########## Executing Genetic Generation: 400 ##########
########## Executing Genetic Generation: 450 ##########
########## Executing Genetic Generation: 500 ##########
########## Executing Genetic Generation: 550 ##########
########## Executing Genetic Generation: 600 ##########
########## Executing Genetic Generation: 650 ##########
########## Executing Genetic Generation: 700 ##########
########## Executing Genetic Generation: 750 ##########
########## Executing Genetic Generation: 800 #######

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1

#Invoking A* with reduced branching factor to understand time and space complexity
number_of_transitions = 2
initial = State(supply, tents_requirement, 0, 0)
generate_state_graph(initial)
invoke_a_star(initial)


In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2



tc = NUM_TRIPS*NUM_PLANS*40

# Time complexity
print(f"Time Complexity is of O({tc})")
# Space Complexity
print(f"Space  Complexity is of O({tc})")


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

**Comparison** : A* tries to attain an objective by finding the optimal path but needs the right heuristic to be defined. On the other hand, since Genetic Algorithm is already a meta heuristic, it can search through the random search space and find the optimal solution with fitness function being the objective.  